## Build BreastCancerNet Convolutional Neural Network (CNN) in PyTorch

The next step is to build CancerNet convolutional neural network the PyTorch way. I took the breast cancer prediction CNN map from here - https://pyimagesearch.com/wp-content/uploads/2019/02/cancernet_arch_raw.png. 

First PyTorch imports are listed on *Lines 2-

Let’s go ahead and define the BreastCancerNet class on Line 12 and then proceed to build it on Line 14.

The build  method requires four parameters:

width , height , and depth : Here we specify the input image volume shape to our network, where depth  is the number of color channels each image contains.

classes : The number of classes our network will predict (for BreastCancerNet , it will be 2 ).
We go ahead and initialize our model  on Line 17 and subsequently, specify our inputShape  (Line 18). In the case of using TensorFlow as our backend, we’re now ready to add layers.

Other backends that specify "channels_first"  require that we place the depth  at the front of the inputShape  and image dimensions following (Lines 23-24).

We use the Sequential API to build BreastCancerNet and Conv2D to implement depthwise convolutions. The class BreastCancerNet has a static method build that takes four parameters- width and height of the image, its depth (the number of color channels in each image), and the number of classes the network will predict between, which, for us, is 2 (0 and 1).

In this method, we initialize model and shape. When using channels_first, we update the shape and the channel dimension.

Now, we’ll define three DEPTHWISE_CONV => RELU => POOL layers; each with a higher stacking and a greater number of filters. The softmax classifier outputs prediction percentages for each class. In the end, we return the model.

Three DEPTHWISE_CONV => RELU => POOL  blocks are defined here with increasing stacking and number of filters. I’ve applied BatchNormalization  and Dropout  as well.

Let’s append our fully connected head:

Our FC => RELU  layers and softmax classifier make the head of the network.

The output of the softmax classifier will be the prediction percentages for each class our model will predict.

Finally, our model  is returned to the training script.

In [1]:
# import the necessary packages 

#container
from torch.nn import Sequential
from torch.nn import Module

#convolution layers
from torch.nn import Conv2d

#pooling layers
from torch.nn import MaxPool2d

#Activations
from torch.nn import ReLU
from torch.nn import Softmax

#normalization layers
from torch.nn import BatchNorm2d

#Linear Layers
from torch.nn import Linear

#Dropout layers
from torch.nn import Dropout

#loss function 
from torch.nn import CrossEntropyLoss 
#This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class.

#Utility functions
from torch.nn import Flatten


In [3]:

#Build CNN BreastCancerNet in PyTorch
#Adopted from CancerNet model in Keras by DataFlair Team

class BreastCancerNet(Module):
    def __init__(self):
        super(BreastCancerNet, self).__init__()
       
    def build(width, height, depth, classes):
        model=Sequential()
        shape=(height, width, depth)
        
        if K.image_data_format() == 'channels_first':
            shape=(depth,height,width)
        
       # Declare all the layers for feature extraction
        self.features = Sequential(Conv2d(in_channels=3,
                                          out_channels=32,
                                          kernel_size = (3, 3), 
                                          padding=1, input_shape=shape),
                                   ReLU(inplace=True),
                                   BatchNorm2d(32),
                                   MaxPool2d(3,3),
                                   Dropout(0.23),
                                   Conv2d(in_channels=32, 
                                          out_channels=64, 
                                          kernel_size = (3, 3), 
                                          padding=1),
                                   ReLU(inplace=True),
                                   BatchNorm2d(64),
                                   MaxPool2d(3,3),
                                   Dropout(0.25),
                                   Conv2d(in_channels=64, 
                                          out_channels=128, 
                                          kernel_size = (3, 3), 
                                          padding=1),
                                   ReLU(inplace=True),
                                   BatchNorm2d(128),
                                   MaxPool2d(3,3),
                                   Dropout(0.25))
        
                                   
        # Declare all the layers for classification (fully connected layers)
        self.classifier = Sequential(
            Linear(128*6*6, 6912),
            ReLU(inplace=True),
            Dropout(0.5),
            Linear(6912, 6912),
            ReLU(inplace=True),
            Linear(6912, 2)
        
        #self._initialize_weights()
                 
    def  forward(self, x): 
                 #Apply the feature extract in the input
                 x = self.features(x)
                 #squeeze the three spatial dimenstions in one 
                 x = x.view(-1, 128*6*6)
                 x = self.classifier(x)
                 return x
